In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
titanic_df = pd.read_csv("./csv_data/titanic_train.csv")
titanic_df.head()

In [ ]:
print("### train 데이터 정보 ### \n")
print(titanic_df.info())    #891 개 중에 null 갯수 알 수 있음

In [ ]:
titanic_df.isnull().sum()

In [ ]:
titanic_df.describe()

In [ ]:
titanic_df["Age"].fillna(titanic_df["Age"].mean(), inplace=True)
titanic_df["Cabin"].fillna("N", inplace=True)
titanic_df["Embarked"].fillna("N", inplace=True)    # 전처리, 데이터의 특성 숙지

In [ ]:
print(f"\n {titanic_df.info()} \n {titanic_df.isnull().sum()}")

In [ ]:
print(f"\n Sex 값 분포 : \n {titanic_df['Sex'].value_counts()}")
print(f"\n Cabin 값 분포 : \n {titanic_df['Cabin'].value_counts()}")
print(f"\n Embarked 값 분포 : \n {titanic_df['Embarked'].value_counts()}")

In [ ]:
titanic_df["Cabin"] = titanic_df["Cabin"].str[:1]   # 첫글자만 따서 저장
print(titanic_df["Cabin"].head())

In [ ]:
titanic_df["Cabin"].value_counts()

In [ ]:
titanic_df.groupby(["Sex", "Survived"])["Survived"].count()

In [ ]:
sns.barplot(x="Sex", y="Survived", data=titanic_df) # 비율로 보여줌

In [ ]:
sns.barplot(x="Pclass", y="Survived", hue="Sex", data=titanic_df)
# hue 는 각각 분할하는 것

In [ ]:
# 입력 age에 따라 구분값을 반환하는 함수 설정. DataFrame 의 apply lambda 식에 사용.
def get_category(age) :
    cat = ''
    if age <= -1 : cat = "Unknown"
    elif age <= 5 : cat = "Baby"
    elif age <= 12 : cat = "Child"
    elif age <= 18 : cat = "Teenager"
    elif age <= 25 : cat = "Student"
    elif age <= 35 : cat = "Young Adult"
    elif age <= 60 : cat = "Adult"
    else : cat = "Elderly"
    
    return cat

In [ ]:
# 막대그래프의 크기 figure를 더 크게 설정
plt.figure(figsize=(10,6))

# X 축의 값을 순차적으로 표시하기 위한 설정  /  sns.barplot(order=)
group_names = ["Unknown", "Baby", "Child", "Teenager", "Student", "Young Adult", "Adult", "Elderly"]

# lambda 식에 위에서 생성한 get_category() 함수를 반환값으로 지정.
# get_category(X)는 입력값으로 "Age" 컬럼값을 받아서 해당하는 cat 반환
titanic_df["Age_cat"] = titanic_df["Age"].apply(lambda x : get_category(x))
sns.barplot(x="Age_cat", y="Survived", hue="Sex", data=titanic_df, order=group_names)
titanic_df.drop("Age_cat", axis=1, inplace=True)    # 한번 하고 없앰

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# 스트링으로 표현된 데이터들 넘버링  /  레이블 인코딩
def encode_features(dataDF) :
    features = ["Cabin", "Sex", "Embarked"]
    for feature in features :
        le = LabelEncoder()
        le = le.fit(dataDF[feature])
        dataDF[feature] = le.transform(dataDF[feature])

    return dataDF

titanic_df = encode_features(titanic_df)
titanic_df.head()

### 전처리 과정을 함수로 리팩토링 → 학습 데이터에 적용하여 쉽게 처리

In [ ]:
# Null 처리 함수
def fillna(df) :
    df["Age"].fillna(df["Age"].mean(), inplace=True)
    df["Cabin"].fillna("N", inplace=True)
    df["Embarked"].fillna("N", inplace=True)
    df["Fare"].fillna(0, inplace=True)
    return df

In [ ]:
# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df) :
    df.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)
    return df

In [ ]:
# 레이블 인코딩 수행
def format_features(df) :
    df["Cabin"] = df["Cabin"].str[:1]
    features = ["Cabin", "Sex", "Embarked"]
    for feature in features :
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

In [ ]:
# 앞에서 설정한 Data Processing 함수 호출
def transform_features(df) :
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [ ]:
# 클래스 만들기
class DtPre :
    
    def __init__(self, df) :
        self.df = df

    def fillna(self) :
        self.df["Age"].fillna(self.df["Age"].mean(), inplace=True)
        self.df["Cabin"].fillna("N", inplace=True)
        self.df["Embarked"].fillna("N", inplace=True)
        self.df["Fare"].fillna(0, inplace=True)
        return self.df

    def drop_features(self) :
        self.df.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)
        return self.df

    def format_features(self) :
        self.df["Cabin"] = self.df["Cabin"].str[:1]
        features = ["Cabin", "Sex", "Embarked"]
        for feature in features :
            le = LabelEncoder()
            le = le.fit(self.df[feature])
            self.df[feature] = le.transform(self.df[feature])
        return self.df
    
    def transform_features(self) :
        self.df = fillna(self.df)
        self.df = drop_features(self.df)
        self.df = format_features(self.df)
        return self.df

### 학습수행

In [ ]:
titanic_df = pd.read_csv("./csv_data/titanic_train.csv")
y_titanic_df = titanic_df["Survived"]   # 레이블 데이터 셋 추출
X_titanic_df = titanic_df.drop("Survived", axis=1)  # 피쳐 데이터 셋에서 레이블셋은 삭제

X_titanic_df = transform_features(X_titanic_df) # 만들어둔 전처리 함수 적용

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,
                                                    test_size=0.2, random_state=121)
# 학습 데이터 셋 분할

#### 디시전 트리, 랜덤 포레스트, 로지스틱 회귀 모형 이용

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 결정트리, Random Forest, 로지스틱 회귀를 위한 Classifier
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression()

# 디시전 트리 모형으로 검증
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print(f"DecisionTreeClassifier 정확도 : {accuracy_score(y_test, dt_pred) : .4f}")

# 랜덤 포레스트 모형으로 검증
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print(f"RandomForestClassifier 정확도 : {accuracy_score(y_test, rf_pred) : .4f}")

# 로지스틱 모형으로 검증
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print(f"LogisticRegression 정확도 : {accuracy_score(y_test, lr_pred) : .4f}")

#### KFold 모형 사용

In [ ]:
from sklearn.model_selection import KFold

def exec_kfold(clf, folds=5) :
    # 폴드 세트가 5개인 KFold객체를 생성, 폴드 수만큼 예측결과 저장을 위해 리스트 객체 생성
    kfold = KFold(n_splits=folds)
    scores = []

    # KFold 교차 검증 수행
    for iter_count , (train_index, test_index) in enumerate(kfold.split(X_titanic_df)) :
        # X_titanic_df 데이터에서 교차 검증별로 학습과 검증 데이터를 가리키는 index 형성
        X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
        y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]

        # Classifier 검증
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print(f"교차검증 {iter_count} 정확도 : {accuracy : .4f}")

    # 5개 fold 에서의 평균 정확도 계산
    mean_score = np.mean(scores)
    print(f"평균 정확도 : {mean_score:.4f}")

# exec_kfold 호출  /  위에서 설정한 디시전 트리 모형 가져옴
exec_kfold(dt_clf, folds=5)

#### Cross_val_score 사용

In [ ]:
from sklearn.model_selection import cross_val_score

# 위에서 설정한 디시전 트리 모형에 대해 피쳐, 레이블 값 넣어서 5회 교차검증
scores = cross_val_score(dt_clf, X_titanic_df, y_titanic_df, cv=5)
for iter_count, accuracy in enumerate(scores) :
    print(f"교차검증 {iter_count} 정확도 : {accuracy:.4f}")

print(f"평균 정확도 : {np.mean(scores):.4f}")

#### GridSearchCV 모형 사용

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {"max_depth" : [2, 3, 5, 10],      # 최대 깊이
              "min_samples_split" : [2, 3, 5],  # 분할되기 위해 노드가 가져야 하는 최소 샘플 수
              "min_samples_leaf" : [1, 5, 8]}   # 리프 노드가 가지고 있어야 할 최소 샘플 수

# 위에서 미리 분할해 놓은 훈련 세트 / test_train_split
grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring="accuracy", cv=5)
grid_dclf.fit(X_train, y_train)

print(f"GridSearchCV 최적 하이퍼 파라미터 : {grid_dclf.best_params_}")
print(f"GridSearchCV 최고 정확도 : {grid_dclf.best_score_:.4f}")
best_dclf = grid_dclf.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행
dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpredictions)
print(f"테스트 세트에서의 DecisionTreeClassifier 정확도 : {accuracy:.4f}")